In [1]:
import boto3
from IPython.display import HTML, display, Image as IImage
from PIL import Image, ImageDraw, ImageFont
import time
import os

In [2]:
session = boto3.session.Session()
region = session.region_name

print('session: ', session)
print('region: ', region)

session:  Session(region_name='ap-northeast-2')
region:  ap-northeast-2


In [3]:
rekognition = boto3.client('rekognition')
s3_client = boto3.client('s3')

In [4]:
bucket_nm = 'aws-rek-immersionday-' + region
prefix = 'media/object-detection/'

print('bucket name: ', bucket_nm)
print('prefix: ', prefix)

bucket name:  aws-rek-immersionday-ap-northeast-2
prefix:  media/object-detection/


In [5]:
!mkdir mltmp
temp_folder = 'm1tmp/'

mkdir: cannot create directory ‘mltmp’: File exists


### Detect Objects in image

In [6]:
bucket_list = s3_client.list_objects(
    Bucket=bucket_nm,
    Prefix=prefix
)

In [7]:
image_prefix_list = []

for obj in bucket_list['Contents'][1:]:
    print('file name: ', obj['Key'])
    obj_path = obj['Key']
    image_prefix_list.append(obj_path)

file name:  media/object-detection/car1.png
file name:  media/object-detection/car2.png
file name:  media/object-detection/car3.png


In [8]:
print('image_path: ', image_prefix_list)

image_path:  ['media/object-detection/car1.png', 'media/object-detection/car2.png', 'media/object-detection/car3.png']


In [9]:
# s3 bucket에 저장한 자동차 이미지 불러오기

for image_prefix in image_prefix_list:
    display(
        IImage(
            url=s3_client.generate_presigned_url(
                'get_object',
                Params={
                    'Bucket': bucket_nm,
                    'Key': image_prefix
                }
            )
        )
    )

### Call Rekognition to detect objects in the image

In [10]:
detect_labels_response_list = []

for image_prefix in image_prefix_list:
    print('bucket name: ', bucket_nm)
    print('image prefix: ', image_prefix)
    detect_labels_response = rekognition.detect_labels(
        Image={
            'S3Object': {
                'Bucket': bucket_nm,
                'Name': image_prefix
            }
        }
    )
    detect_labels_response_list.append(detect_labels_response)

bucket name:  aws-rek-immersionday-ap-northeast-2
image prefix:  media/object-detection/car1.png
bucket name:  aws-rek-immersionday-ap-northeast-2
image prefix:  media/object-detection/car2.png
bucket name:  aws-rek-immersionday-ap-northeast-2
image prefix:  media/object-detection/car3.png


In [11]:
# Show JSON response returned by Rekognition Labels API

flaggedObjects = ['Car']

for label in detect_labels_response_list[0]['Labels']:
    if (label['Name'] in flaggedObjects):
        print("Detected unsafe object: ")
        print("- {} (Confidence: {})".format(label['Name'], label['Confidence']))
        print("  - Parents: {}".format(label["Parents"]))

Detected unsafe object: 
- Car (Confidence: 99.67633056640625)
  - Parents: [{'Name': 'Vehicle'}, {'Name': 'Transportation'}]


In [13]:
flaggedObjects = ['Car']

for image_prefix, detect_labels_response in zip(image_prefix_list, detect_labels_response_list):
    print("Image Prefix: ", image_prefix)
    
    # Show Image
    display(
        IImage(
            url=s3_client.generate_presigned_url(
                'get_object',
                Params={
                    'Bucket': bucket_nm,
                    'Key': image_prefix
                }
            )
        )
    )
    
    # Find Label is Car and show confidence value
    for label in detect_labels_response['Labels']:
        if label['Name'] in flaggedObjects:
            print("Detected unsafe object: ")
            print("- {} (Confidence: {})".format(label["Name"], label["Confidence"]))
            print("  - Parents: {}".format(label["Parents"]))

Image Prefix:  media/object-detection/car1.png


Detected unsafe object: 
- Car (Confidence: 99.67633056640625)
  - Parents: [{'Name': 'Vehicle'}, {'Name': 'Transportation'}]
Image Prefix:  media/object-detection/car2.png


Detected unsafe object: 
- Car (Confidence: 99.85629272460938)
  - Parents: [{'Name': 'Vehicle'}, {'Name': 'Transportation'}]
Image Prefix:  media/object-detection/car3.png


Detected unsafe object: 
- Car (Confidence: 99.13117980957031)
  - Parents: [{'Name': 'Vehicle'}, {'Name': 'Transportation'}]
